In [ ]:
import torch
import torchvision
import torchvision.transforms as transforms
from torch.nn import Linear, ReLU, Sequential, Conv2d, MaxPool2d, Module,Flatten,  BatchNorm2d, CrossEntropyLoss
from torch.optim import Adam, SGD
import torch.optim as optim
import numpy as np
from torch.optim.lr_scheduler import *
from tqdm import tqdm
import torch.nn.functional as F
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Specify oldClasses Below


In [ ]:
oldClasses = [0,1,2,3,4,5]

# Below trainset and testset are both for K classes(part of all labels)

In [ ]:
transform = transforms.Compose(
    [transforms.ToTensor()])

batch_size = 64

trainsetOld = torchvision.datasets.FashionMNIST(root='./data', train=True,
                                        download=True, transform=transform)
trainloaderOld = torch.utils.data.DataLoader(trainsetOld, batch_size=batch_size,
                                          shuffle=True, num_workers=2)

testsetOld = torchvision.datasets.FashionMNIST(root='./data', train=False,
                                       download=True, transform=transform)
testloaderOld = torch.utils.data.DataLoader(testsetOld, batch_size=batch_size,
                                         shuffle=False, num_workers=2)
print(np.unique(testsetOld.targets))

[0 1 2 3 4 5 6 7 8 9]


In [ ]:
idx = torch.zeros(trainsetOld.targets.shape, dtype=torch.bool)
for k in oldClasses:
  idx = idx | (trainsetOld.targets==k)
trainsetOld.targets = trainsetOld.targets[idx]
trainsetOld.data = trainsetOld.data[idx]

## Modify Test data
idx = torch.zeros(testsetOld.targets.shape, dtype=torch.bool)
for k in oldClasses:
  idx = idx | (testsetOld.targets==k)
testsetOld.targets = testsetOld.targets[idx]
testsetOld.data = testsetOld.data[idx]

In [ ]:
print(trainsetOld.data.shape, trainsetOld.targets.shape)
print(testsetOld.data.shape, testsetOld.targets.shape)
print(np.unique(trainsetOld.targets))

torch.Size([36000, 28, 28]) torch.Size([36000])
torch.Size([6000, 28, 28]) torch.Size([6000])
[0 1 2 3 4 5]


In [ ]:
class CNN(Module):
  def __init__(self):
        super().__init__()
        self.network = Sequential(
            Conv2d(1, 32, kernel_size=3, padding=1),
            ReLU(),
            Conv2d(32, 64, kernel_size=3, padding=1),
            ReLU(),
            MaxPool2d(2, 2), # output: 64 x 16 x 16

            Conv2d(64, 128, kernel_size=3, padding=1),
            ReLU(),
            Conv2d(128, 128, kernel_size=3,padding=1),
            ReLU(),
            MaxPool2d(2, 2), # output: 128 x 8 x 8

            Conv2d(128, 256, kernel_size=3,padding=1),
            ReLU(),
            Conv2d(256, 256, kernel_size=3,padding=1),
            ReLU(),
            MaxPool2d(2, 2), # output: 256 x 4 x 4

            Flatten(), 
            Linear(1*2304, 1024),
            ReLU(),
            Linear(1024, 512),
            ReLU(),
            Linear(512, len(oldClasses)))
  def forward(self, xb):
      return self.network(xb)

In [ ]:
model = CNN().cuda()
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)
scheduler = MultiStepLR(optimizer, milestones=[30,80], gamma=0.1)

In [ ]:
def train(model, train_loader, optimizer):
    model = model.to(device)
    model.train()
    num_examples = 0
    correct = 0
    train_loss = 0
    for batch_idx, (data, target) in tqdm(enumerate(train_loader)):
        data, target = data.to(device), target.to(device)
        output = model(data)
        #print(data.shape,output.shape, target.shape)
        loss = F.cross_entropy(output, target)
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()
        pred = output.max(1, keepdim=True)[1]
        correct += pred.eq(target.view_as(pred)).sum().item()
        train_loss += F.cross_entropy(output, target, reduction='sum').item()
        num_examples += len(data)

    train_loss /= num_examples
    train_acc = 100. * correct / num_examples
    print(f'Train set: Average loss: {train_loss:.4f}, '
          f'Accuracy: {correct}/{num_examples} ({train_acc:.2f}%)')
    return train_loss, train_acc
def test(model, test_loader):
    device = next(model.parameters()).device
    
    model.eval()
    num_examples = 0
    test_loss = 0
    correct = 0

    with torch.no_grad():
        for data, target in tqdm(test_loader):
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.cross_entropy(output, target, reduction='sum').item()
            pred = output.max(1, keepdim=True)[1]
            correct += pred.eq(target.view_as(pred)).sum().item()
            num_examples += len(data)

    test_loss /= num_examples
    test_acc = 100. * correct / num_examples

    print(f'Test set: Average loss: {test_loss:.4f}, '
          f'Accuracy: {correct}/{num_examples} ({test_acc:.2f}%)')

    return test_loss, test_acc

In [ ]:
from torchsummary import summary
summary(model, (1,28,28))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 32, 28, 28]             320
              ReLU-2           [-1, 32, 28, 28]               0
            Conv2d-3           [-1, 64, 28, 28]          18,496
              ReLU-4           [-1, 64, 28, 28]               0
         MaxPool2d-5           [-1, 64, 14, 14]               0
            Conv2d-6          [-1, 128, 14, 14]          73,856
              ReLU-7          [-1, 128, 14, 14]               0
            Conv2d-8          [-1, 128, 14, 14]         147,584
              ReLU-9          [-1, 128, 14, 14]               0
        MaxPool2d-10            [-1, 128, 7, 7]               0
           Conv2d-11            [-1, 256, 7, 7]         295,168
             ReLU-12            [-1, 256, 7, 7]               0
           Conv2d-13            [-1, 256, 7, 7]         590,080
             ReLU-14            [-1, 25

In [ ]:
Epochs = 100
train_loss_lst=[]
train_acc_lst=[]
val_loss_lst=[]
val_acc_lst=[]
for epoch in range(0, Epochs):
    print(f"\nEpoch: {epoch}")
    train_loss, train_acc = train(model, trainloaderOld, optimizer)
    test_loss, test_acc = test(model, testloaderOld)
    train_loss_lst.append(train_loss)
    train_acc_lst.append(train_acc)
    val_loss_lst.append(test_loss)
    val_acc_lst.append(test_acc)
    print(train_loss_lst[-1])
    print(train_acc_lst[-1])
    print(val_loss_lst[-1])
    print(val_acc_lst[-1])
    scheduler.step()


Epoch: 0


563it [00:18, 30.53it/s]


Train set: Average loss: 1.7918, Accuracy: 6017/36000 (16.71%)


100%|██████████| 94/94 [00:01<00:00, 65.48it/s]

Test set: Average loss: 1.7916, Accuracy: 1000/6000 (16.67%)
1.7918403951856825
16.71388888888889
1.7916202672322592
16.666666666666668

Epoch: 1


...

Train set: Average loss: 0.2375, Accuracy: 32784/36000 (91.07%)


100%|██████████| 94/94 [00:01<00:00, 65.29it/s]

Test set: Average loss: 0.2585, Accuracy: 5414/6000 (90.23%)
0.23745366793870926
91.06666666666666
0.2584817158381144
90.23333333333333

Epoch: 20



493it [00:15, 31.19it/s]


KeyboardInterrupt: ignored

#Get new model, only modify last layer

In [ ]:
import copy
newModel = copy.deepcopy(model)

In [ ]:
newModel

CNN(
  (network): Sequential(
    (0): Conv2d(1, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
    (2): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU()
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU()
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU()
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU()
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU()
    (14): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (15): Flatten(start_dim=1, end_dim=-1)
    (16): Linear(in_features=2304, out_features=1024, bias=True)
    (17): ReLU()
    (18): Linear(in_features=1024, o

In [ ]:
newModel.network[-1].out_features = 10

In [ ]:
newModel

CNN(
  (network): Sequential(
    (0): Conv2d(1, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
    (2): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU()
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU()
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU()
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU()
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU()
    (14): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (15): Flatten(start_dim=1, end_dim=-1)
    (16): Linear(in_features=2304, out_features=1024, bias=True)
    (17): ReLU()
    (18): Linear(in_features=1024, o

In [ ]:
trainsetFull = torchvision.datasets.FashionMNIST(root='./data', train=True,
                                        download=True, transform=transform)
trainloaderFull = torch.utils.data.DataLoader(trainsetFull, batch_size=batch_size,
                                          shuffle=True, num_workers=2)

testsetFull = torchvision.datasets.FashionMNIST(root='./data', train=False,
                                       download=True, transform=transform)
testloaderFull = torch.utils.data.DataLoader(testsetFull, batch_size=batch_size,
                                         shuffle=False, num_workers=2)
print(trainsetFull.data.shape, trainsetFull.targets.shape)
print(testsetFull.data.shape, testsetFull.targets.shape)

In [ ]:
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)
scheduler = MultiStepLR(optimizer, milestones=[30,80], gamma=0.1)

In [ ]:
Epochs = 100
train_loss_lst=[]
train_acc_lst=[]
val_loss_lst=[]
val_acc_lst=[]
for epoch in range(0, Epochs):
    print(f"\nEpoch: {epoch}")
    train_loss, train_acc = train(model, trainloaderFull, optimizer)
    test_loss, test_acc = test(model, testloaderFull)
    train_loss_lst.append(train_loss)
    train_acc_lst.append(train_acc)
    val_loss_lst.append(test_loss)
    val_acc_lst.append(test_acc)
    print(train_loss_lst[-1])
    print(train_acc_lst[-1])
    print(val_loss_lst[-1])
    print(val_acc_lst[-1])
    scheduler.step()